# Artificial Intelligence :  Computer Assignment 1  
> __Morteza Nouri, 810198481__

## Goals:
- Learning informed search algorithms (A*)
- Learning uninformed search algorithms (BFS, IDS)
- How to formulate problems (Abstraction)
- Pros and Cons of each algorithm

## Description:
> In this project, we have a 2D grid that has some characters and rules. the hero of this game (Gandalf) should transfer all of his friends to their specific castles, But there are guards (Orks) that, each have their own territory, and Gandalf must carry out his mission without being arrested. After that, Gandalf should go to his goal home, Gandor. Gandalf only moves left, right, up and down. Our task is to find a way for Gandalf to succeed in her mission.



## Modeling:
 __States:__ Each state represented by a tuple:  $(currentPosition, crossingFriend, transferedFriends, risk) $ 
 > _currentPosition_: current position of Gandalf --> (x, y). <br>
 > _crossingFriend_: a friend carried by Gandalf and crosssing --> ith friend. if no one crossing --> -1 <br>
 > _transferedFriends_: a binary string of lenght friends that shows if ith friend is in his castle or not --> 00010. <br>
 > _risk_: number of Gandalf appearences in Ork area, if the risk was greater than the rank of an Ork then Gandalf dies.
 
__Initial State:__ $(startPosition, -1, '000..0', 0)$ <br>
__Actions:__ moving Left, Right, Up and Down, carring and crossing when Gandalf and friend have same position. <br>
__Transition model:__ Gandalf goes left, right, up and down to find a friend then carring him to his castle. If Gandalf is in Ork territory, the number of risks increased based on Ork rank.
As soon as Gandalf leaves the territory, risk becaomes zero. <br>
__Goal State:__ $(endPosition, -1, '111..1', 0)$ <br>
__Action Cost:__ each actions costs 1.


### Reading Data 

In This part we read test file and based on that we construct the game grid.


In [1]:
test = open('sample_testcases/test_02.txt', 'r')

lines = []
for line in test:
    lines.append(line.rstrip('\n'))


ORK = '-' # Ork symbol in grid

# getting data to construct grid with n rows and m columns

n, m = map(int, lines[0].split())
grid = [[0 for i in range(m)] for j in range(n)]

# getting start and end position of Gandalf

startPos = tuple(map(int, lines[1].split()))
endPos = tuple(map(int, lines[2].split()))


k, l = map(int, lines[3].split())

# getting orks positions

orks = []
for i in range(k):
    ork = tuple(map(int, lines[4 + i].split()))
    orks.append(ork)
    grid[ork[0]][ork[1]] = ORK

# getting friends positions

friends = []
for i in range(l):
    friend = tuple(map(int, lines[4 + k +i].split()))
    friends.append(friend)

# getting castles positions for each friend.
    
castles = []
for i in range(l):
    castle = tuple(map(int, lines[4 + k + l + i].split()))
    castles.append(castle)
    
#determining territory of each ork in grid
ownerOrk = dict()
for ork in orks:
    x, y, rank = ork[0], ork[1], ork[2]
    for i in range(max(0, x - rank), min(n - 1, x + rank)  + 1):
        for j in range(max(0, y - rank), min(m - 1, y + rank) + 1):
            if abs(x - i) + abs(y - j) <= rank and (i,j) != (x, y):
                grid[i][j] = rank
                ownerOrk[(i, j)] = ork


## Approach 1 - BFS (Breadth First Search):


In [2]:
#(currentPos, crossingFriend, transferedFriends, risk)

startState = (startPos, -1, '0' * len(friends), 0)
goalState = (endPos, -1, '1' * len(friends), 0)

In [3]:
import time
    

dRow = [-1, 0, 1, 0]
dCol = [0, 1, 0, -1]
direction = {0 : 'U', 1 : 'R', 2 : 'D', 3 : 'L'}

path = []
moves = dict()
parent = dict()
explored = dict()

def BFS():
    q = []
    q.append(startState)
    explored[startState] = 1

    while len(q):
        curState = q.pop(0)
        if curState == goalState:
            break
        
        curPos = curState[0]
        crossing = curState[1]
        transfered = curState[2]
        
        #In this part Gandalf put friend in his castle.
        
        if crossing != -1:
            if curPos == castles[crossing]:
                transfered = transfered[:crossing] + '1' + transfered[crossing + 1:]
                crossing = -1
        
        #In this part Gandalf pick a friend
        
        for i in range(len(friends)):
            if transfered[i] == '1' or crossing == i:
                continue; 
            friendPos = friends[i]
            if friendPos == curPos:
                crossing = i
        
        #Gandalf goes left, right, up, down
        
        for i in range(4):
            px, py = curPos[0], curPos[1] 
            nx = px + dRow[i]
            ny = py + dCol[i]
            if nx >= 0 and nx <= n-1 and ny >= 0 and ny <= m-1:
                curOrk = (-1, -1) if not (px, py) in ownerOrk else ownerOrk[(px, py)]
                risk = curState[3]
                if risk == 0 and grid[nx][ny]:
                    risk += 1
                elif risk and (nx, ny) in ownerOrk and ownerOrk[(nx, ny)] == curOrk:
                    risk += 1
                else:
                    risk = 0
                if grid[nx][ny] != ORK and risk <= grid[nx][ny]:
                    nextState = ((nx, ny), crossing, transfered, risk)
                    if not nextState in explored:
                        q.append(nextState)
                        explored[nextState] = 1
                        parent[nextState] = curState
                        moves[nextState] = direction[i]


In [4]:
def constructPath(curState):
    if curState == startState:
        return
    path.append(moves[curState])
    constructPath(parent[curState])

In [5]:
stime = time.time()
BFS()
print(f"Time spent: {(time.time() - stime) * 1000} ms")
constructPath(goalState)
print(f"Length of the path: {len(path)}")
print(*path[::-1])
print(f"number of explored states: {len(explored)}")

Time spent: 11.435508728027344 ms
Length of the path: 34
R R R R R R R D D D D L L L L D L L L R D D U U R U R R R R D R D D
number of explored states: 420


### BFS Results:

| TestNo | Path Lenght | Explored States | Avg Exec Time (ms) |
| :-: | :-: | :-: | :-: |
| 1 | 48 | 8475 | 37.460962931 |
| 2 | 52 | 1888 | 11.794408162 |
| 3 | 34 | 420 | 1.829067866 |
| 4 | 66 | 6391 | 32.912572225 |

<center> *Average Execution Time is based on running a test 3 times. </center>

>__Advantages of BFS:__ <br>
> - Complete: It finds a solution if there exists.<br>
> - Optimal: If there are various solutions, BFS finds the least costly one.<br>

>__Disadvantages of BFS:__<br>
> - Memory Constraint: It stores all of the explored states.<br>
> - Time Consumption: if solution is far away, it consumes time.<br>
> - Cost Constraint: BFS finds the path with the fewest steps, but does not always find the cheapest path if action cost is not 1.



## Approach 2 - IDS (Iterative Deepening Search):

IDS uses DFS as subroutine and checks for the path of lenght 0, 1, 2, ... in order to get the solution path.

In [6]:
path = []
moves = dict()
parent = dict()
dist = dict()

def DFS(curState, parState, goalState, depth, visited):
    
    visited.add(curState)
    parent[curState] = parState
    if curState != parState:
        dist[curState] = dist[parState] + 1
    
    if curState == goalState:
        return True
    
    if depth <= 0:
        return False
            
    
    curPos = curState[0]
    crossing = curState[1]
    transfered = curState[2]

    #In this part Gandalf put friend in his castle.
    
    if crossing != -1:
        if curPos == castles[crossing]:
            transfered = transfered[:crossing] + '1' + transfered[crossing + 1:]
            crossing = -1
            
    #In this part Gandalf pick a friend
    
    for i in range(len(friends)):
        if transfered[i] == '1' or crossing == i:
            continue; 
        friendPos = friends[i]
        if friendPos == curPos:
            crossing = i
            
    #Gandalf goes left, right, up, down        
    
    for i in range(4):
        px, py = curPos[0], curPos[1] 
        nx = px + dRow[i]
        ny = py + dCol[i]
        if nx >= 0 and nx <= n-1 and ny >= 0 and ny <= m-1:
            curOrk = (-1, -1) if not (px, py) in ownerOrk else ownerOrk[(px, py)]
            risk = curState[3]
            if risk == 0 and grid[nx][ny]:
                risk += 1
            elif risk and (nx, ny) in ownerOrk and ownerOrk[(nx, ny)] == curOrk:
                risk += 1
            else:
                risk = 0
            if grid[nx][ny] != ORK and risk <= grid[nx][ny]:
                nextState = ((nx, ny), crossing, transfered, risk)
                if not nextState in visited:
                    moves[nextState] = direction[i]
                    if DFS(nextState, curState,goalState, depth - 1, visited):
                        return True
                elif dist[nextState] > dist[curState] + 1:
                    moves[nextState] = direction[i]
                    if DFS(nextState, curState, goalState, depth - 1, visited):
                        return True
    return False


In [7]:
def IDS(startState, goalState, maxDepth):
    dist[startState] = 0
    for i in range(maxDepth + 1):
        visited = set()
        if DFS(startState, startState, goalState, i, visited):
            return len(visited)
    return 0

In [8]:
stime = time.time()
states = IDS(startState, goalState, 100)
print(f"Time spent: {(time.time() - stime) * 1000} ms")
print(f"Length of the path: {dist[goalState]}")
constructPath(goalState)
print(*path[::-1])
print(f"number of explored states: {states}")

Time spent: 179.24022674560547 ms
Length of the path: 34
R R R R R R R D D D D L L L L D L L L R D D U U R U R R R R D R D D
number of explored states: 289


### IDS Results:

| TestNo | Path Lenght | Explored States | Avg Exec Time (ms) |
| :-: | :-: | :-: | :-: |
| 1 | 48 | 7846 | 791.612148285 |
| 2 | 52 | 1867 | 1711.511691411 |
| 3 | 34 | 289 | 96.397558848 |
| 4 | 66 | 6329 | 9940.868139267 |

<center> *Average Execution Time is based on running a test 3 times. </center>
<center> *Important: Explored states in this part shows just states which visited in solution path.In fact, explored states are much more than what we have shown. </center> 

>__Advantages of IDS:__ <br>
> - Complete
> - Optimal
> - Space Complexity: As we observe in above table, It's obvious that explored states in IDS is smaller than BFS. Actually, IDS inherits optimality of BFS and space complexity of DFS.

>__Disadvantages of IDS:__<br>
> - Repeatedly time and wasted calculations in each depth before reaching the answer depth.



## Approach 3 - (A*):


__Heuristic Function:__ we have 3 different heuristic according to the game state: <br>
*to calculate heuristic value we use Manhattan Distance*: $ manhattan(src, dest) $
> __if all friends have been transferred:__ $ manhattan(GandalfCurrentState, GandalfGoalState) $ <br>
> __if Gandalf currently carrying ith friend:__  $ manhattan(friends[i], castles[i]) + manhattan(castles[i], GandalfGoalState) $ <br>
> __if Gandalf hasn't picked a friend:__ 
>> for each remaining friend calculate: $ manhattan(GandalfCurrentState, friends[i]) + manhattan(friends[i], castles[i]) + manhattan(castles[i], GandalfGoalState) $ <br>
>> return minimum value of the previous section.

$ f(n) = h(n) + g(n)$
- $ f(n) $ : The evaluation function which is the estimated total cost of the path through state n to the goal state. <br>
- $ g(n) $ : cost so far to reach n <br>
- $ h(n) $ : heuristic <br>
*_in each step, we expand state with minimum value of $ f(n) $ , for this purpos we store $ f $'s in min heap data structure._

__Heuristic admissibility__: Because in each step, Gandalf can move only left, right, up and down, according to definition of manhattan distance, it is obvious that, $ h(n) <= h*(n) $, (where $ h*(n) $ is the real cost to reach goal state)

__Heuristic Consistency__: Suppose Gandalf wants to go from A to C, because in each step we can move only left, right, up and down, So $ g(C) = g(A) + 1 $ and by definition of manhattan distance we have $ g(c) + h(c) >= h(A) $


In [9]:
path = []
moves = dict()
parent = dict()
heap = dict()

def manhattan(src, dest):
    return (abs(src[0] - dest[0]) + abs(src[1] - dest[1]))   

def allFriendsTransfered(transfered):
    for c in transfered:
        if c == '0':
            return False
    return True

def minHForRemainingFriends(gSrc, gDest, transfered):
    minH = 0
    for i in range(len(friends)):
        if transfered[i] == '0':
            h = manhattan(gSrc, friends[i]) + manhattan(friends[i], castles[i]) + manhattan(castles[i], gDest)
            minH = h if h > minH else minH
    return minH    

def getHeuristic(gSrc, gDest, crossing, transfered, alpha):
    if allFriendsTransfered(transfered):
        return manhattan(gSrc, gDest) * alpha
    if crossing != -1:
        return (manhattan(friends[crossing], castles[crossing]) + manhattan(castles[crossing], gDest)) * alpha
    else:
        return minHForRemainingFriends(gSrc, gDest, transfered) * alpha

In [10]:
from heapq import heappush, heappop

def Astar(alpha):
    openset = []
    closedset = set()
    startF = 0 + getHeuristic(startState[0], goalState[0], -1, '0' * len(friends), alpha)
    heappush(openset, (startF, startState))
    heap[(startF, startState)] = 1
    
    while openset:
        frontier = heappop(openset)
        del heap[frontier]
        
        if frontier[1] == goalState:
            return len(closedset)

        closedset.add(frontier[1])
        
        curF = frontier[0]          
        curPos = frontier[1][0]
        crossing = frontier[1][1]
        transfered = frontier[1][2]

        #put a friend
        if crossing != -1:
            if curPos == castles[crossing]:
                transfered = transfered[:crossing] + '1' + transfered[crossing + 1:]
                crossing = -1

        # pick a friend
        for i in range(len(friends)):
            if transfered[i] == '1' or crossing != -1:
                continue; 
            friendPos = friends[i]
            if friendPos == curPos:
                crossing = i
        
        
        for i in range(4):
            px, py = curPos[0], curPos[1]
            nx = px + dRow[i]
            ny = py + dCol[i]
            if nx >= 0 and nx <= n-1 and ny >= 0 and ny <= m-1:
                curOrk = (-1, -1) if not (px, py) in ownerOrk else ownerOrk[(px, py)]
                risk = frontier[1][3]
                if risk == 0 and grid[nx][ny]:
                    risk += 1
                elif risk and (nx, ny) in ownerOrk and ownerOrk[(nx, ny)] == curOrk:
                    risk += 1
                else:
                    risk = 0
                if grid[nx][ny] != ORK and risk <= grid[nx][ny]:
                    nextState = ((nx, ny), crossing, transfered, risk)
                    curG = curF - getHeuristic(curPos, goalState[0], crossing, transfered, alpha)
                    nextF = curG + 1 + getHeuristic(nextState[0], goalState[0], crossing, transfered, alpha)
                    if (not nextState in closedset) and (not (nextF, nextState) in heap):
                        parent[nextState] = frontier[1]
                        moves[nextState] = direction[i]
                        heappush(openset, (nextF, nextState))
                        heap[(nextF, nextState)] = 1

In [11]:
stime = time.time()
states = Astar(alpha=1)
print(f"Time spent: {(time.time() - stime) * 1000} ms")
constructPath(goalState)
print(f"Length of the path: {len(path)}")
print(*path[::-1])
print(f"number of explored states: {states}")

Time spent: 22.339820861816406 ms
Length of the path: 34
R R R R R R R L L L L D D D D L D L L R D D U U R U R R R R D R D D
number of explored states: 207


### A* Results:
| TestNo | Path Lenght | Explored States | Avg Exec Time (ms) |
| :-: | :-: | :-: | :-: |
| 1 | 48 | 297 | 9.778181712 |
| 2 | 52 | 1187 | 20.514329274 |
| 3 | 34 | 207 | 2.953767776 |
| 4 | 66 | 3246 | 58.925946554 |

<center> *Average Execution Time is based on running a test 3 times. </center>

>__Advantages of A*:__ <br>
> - Optimal: because our heuristic in consistence it finds the best solution.
> - Fast: as we saw in results, A* explores less states than BFS and IDS.

>__DisAdvantages of A*:__ <br>
> - It's Complete if the branching factor is finite and evey action has fixed cost.(like our problem).
> - It's Speed and Optimality highly depends on choosen heuristic.

## Approach 4 - (Weighted A*)

In this method we inflate our heuristic by $ a > 1 $ and then perform A* search as usual.

### $ a = 2 $

In [12]:
stime = time.time()
states = Astar(alpha=2)
print(f"Time spent: {(time.time() - stime) * 1000} ms")
constructPath(goalState)
print(f"Length of the path: {len(path)}")
print(*path[::-1])
print(f"number of explored states: {states}")


Time spent: 9.900808334350586 ms
Length of the path: 68
R R R R R R R L L L L D D D D L D L L R D D U U R U R R R R D R D D R R R R R R R L L L L D D D D L D L L R D D U U R U R R R R D R D D
number of explored states: 164


#### Results:
| TestNo | Path Lenght | Explored States | Avg Exec Time (ms) |
| :-: | :-: | :-: | :-: |
| 1 | 96 | 67 | 2.550363541 |
| 2 | 104 | 373 | 6.790240606|
| 3 | 68 | 164 | 2.7538675476 |
| 4 | 142 | 2431 | 39.925946554 |

### $ a = 5 $

In [13]:
stime = time.time()
states = Astar(alpha=5)
print(f"Time spent: {(time.time() - stime) * 1000} ms")
constructPath(goalState)
print(f"Length of the path: {len(path)}")
print(*path[::-1])
print(f"number of explored states: {states}")

Time spent: 3.9954185485839844 ms
Length of the path: 102
R R R R R R R L L L L D D D D L D L L R D D U U R U R R R R D R D D R R R R R R R L L L L D D D D L D L L R D D U U R U R R R R D R D D R R R R R R R L L L L D D D D L D L L R D D U U R U R R R R D R D D
number of explored states: 116


#### Results: 
| TestNo | Path Lenght | Explored States | Avg Exec Time (ms) |
| :-: | :-: | :-: | :-: |
| 1 | 144 | 63 | 2.550363541 |
| 2 | 156 | 262 | 6.790240606|
| 3 | 102 | 116 | 1.7538675476 |
| 4 | 210 | 1762 | 42.925946554 |

### Weighted A* Results:
> __Advantages of Weighted A*:__ <br>
> - Complete <br>
> - By increasing the weight, less states explored, So it is fast.<br>

> __Disadvantages of Weighted A*:__ <br>
> - Optimaly is lost and heuristic is not consistent anymore.